In [883]:
# !IPython.load_extensions('calico-document-tools') To construct notebook Table of Contents
import numpy as np
import pandas as pd
import pysal as ps
import scipy
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import minimize

from scipy.stats import beta # Beta probability function
from scipy.special import digamma
from scipy.special import beta as f_beta # Beta raw function

import os

os.chdir('C:/Users/renan/Desktop/inequality-segregation-supplementary-files/')
census_2010 = pd.read_csv('data/std_2010_fullcount.csv', encoding = "ISO-8859-1", sep = ",")
census_2010.head()

,trtid10,state,county,tract,pop10,nhwht10,nhblk10,ntv10,asian10,hisp10,...,a15hsp10,a60hsp10,ageasn10,a15asn10,a60asn10,agentv10,a15ntv10,a60ntv10,globd10,globg10
0,1001020100,AL,Autauga County,Census Tract 201,1912,1601,228,21,16,44,...,14,2,14,4,1,13,1,3,bw,White Black
1,6083002402,CA,Santa Barbara County,Census Tract 24.02,11406,1980,207,54,703,8439,...,2972,414,624,119,75,26,4,3,wha,Dual immig
2,1001020200,AL,Autauga County,Census Tract 202,2170,844,1226,9,13,75,...,14,1,5,0,0,5,3,1,bw,White Black
3,6083002102,CA,Santa Barbara County,Census Tract 21.02,2084,853,24,24,88,1088,...,342,84,61,4,16,11,0,4,wha,Dual immig
4,1001020300,AL,Autauga County,Census Tract 203,3373,2538,668,30,42,87,...,34,9,22,2,7,9,1,3,bw,White Black


In [788]:
def duncan_betamix(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]

    if (Cmix == 1):
        lam = 1 
    else: 
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = sum(lam * (b * beta.cdf(mu, a, b + 1)/(1 - mu) - a * beta.cdf(mu, a + 1, b) / mu) / (a + b))
    return value

In [789]:
duncan_betamix([0.3, 0.7])

0.6657228773303705

In [790]:
duncan_betamix([0.3, 0.7, 0.04, 0.65, 0.7])

0.8075080072970819

In [791]:
duncan_betamix([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.7403564967329468

In [792]:
def theil_betamix_2017(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = 1 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = 1 - sum(lam * (a / (a + b) * digamma(a + 1) - a / (a + b) * digamma(a + b + 1))) / (mu * np.log(mu))
    return value

In [793]:
theil_betamix_2017([0.3, 0.7])

0.5083151198778997

In [794]:
theil_betamix_2017([0.3, 0.7, 0.04, 0.65, 0.7])

0.5348452644472066

In [795]:
theil_betamix_2017([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.6219157169201162

In [796]:
def theil_betamix_2012(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = 1 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = 1 - sum(lam * (a / (a + b) * digamma(a + 1) + b / (a + b) * digamma(b + 1) - digamma(a + b + 1))) / (mu * np.log(mu) + (1 - mu) * np.log(1 - mu))
    return value

In [797]:
theil_betamix_2012([0.3, 0.7])

0.46377929067849466

In [798]:
theil_betamix_2012([0.3, 0.7, 0.04, 0.65, 0.7])

0.6436871528489695

In [799]:
theil_betamix_2012([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.5568732745587515

In [800]:
def gini_betamix(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = np.array([1]) 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))
    

    def ff(x):
        res = np.empty(shape = (Cmix, 2))
        for ii in range(Cmix):
            res[ii,0] = lam[ii] * b[ii] * beta.pdf(x, a[ii], b[ii] + 1) / (a[ii] + b[ii])
            res[ii,1] = lam[ii] * a[ii] * beta.cdf(x, a[ii] + 1, b[ii]) / (a[ii] + b[ii])
        res = np.sum(res, axis = 0) # apply(res, c(2, 3), sum)
        
        return (res[0] * res[1])

    value = 1-(2/(mu*(1-mu)))*(quad(ff,0,.5,limit=100)[0]+quad(ff,.5,1,limit=100)[0])
    return value


In [801]:
gini_betamix([0.3, 0.7])

0.834506614191632

In [802]:
gini_betamix([0.3, 0.7, 0.04, 0.65, 0.7])

0.9342811402444533

In [803]:
gini_betamix([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.891209223701447

In [804]:
def fmn(a,b,m,n):
    value = f_beta(a + n, b + m - n) / f_beta(a, b)
    return value

In [805]:
fmn(0.3, 0.4, 7, 3)

0.0019209261240983927

In [858]:
def llik_cnt(a, b, lam, K, X):
    # a = vector of size C (number of components of the mixture)
    # b = vector of size C (number of components of the mixture)
    # lam = vector of size C (number of components of the mixture)
    # K = scalar; number of indiv by unit
    # X = vector of size K+1; number of units with (j-1) minorities
    
    L = np.array(np.repeat(lam, K+1)).reshape((len(lam), K+1)) # Repeat each element of the list K+1 times, since Python fills rowwisely
    A = np.array(np.repeat(a,   K+1)).reshape((len(a), K+1))
    B = np.array(np.repeat(b,   K+1)).reshape((len(b), K+1))
    
    Kmat = np.array(list(range(K + 1)) * len(lam)).reshape((len(lam), K + 1))

    value = -sum(X * np.log(np.sum(np.multiply(L, fmn(A, B, K, Kmat)), axis = 0))) # multiply element wisely
    return value

In [859]:
np.repeat(lam, len(lam)*(K+1))

array([0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
       0.45, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55,
       0.55, 0.55])

In [863]:
a = [0.3, 0.6]
b = [0.44, 0.9]
lam = [0.45, 0.55]
K = 5
X = [6, 9, 10, 4, 8, 7]
llik_cnt(a, b, lam, K, X)

142.61710934389515

In [1008]:
a = [0.3, 0.6, 0.9, 0.99]
b = [0.44, 0.9, 0.98, 0.01]
lam = [0.1, 0.1, 0.1, 0.7]
K = 5
X = [6, 9, 10, 4, 8, 7]
llik_cnt(a, b, lam, K, X)

172.3409740540395

In [1035]:
def llik_Kvec(a, b, lam, Xmat, Kvec):
    cumsum = 0
    
    for j in list(range(len(Kvec))):
        #print(lam)
        #print(j)
        print({'a': a,
             'b': b,
             'lam': lam,
             'Kvec[j]': Kvec[j],
             'cumsum': cumsum})
        cumsum += llik_cnt(a, b, lam, Kvec[j], Xmat[0:(Kvec[j]+1), j])
        
    
    return cumsum


In [960]:
Kvec = [3, 2]

values = np.linspace(1, 20, num = 20)
Xmat = np.array(values).reshape((int(len(values) / len(Kvec)), len(Kvec)))
Xmat

array([[ 1.,  2.],
       [ 3.,  4.],
       [ 5.,  6.],
       [ 7.,  8.],
       [ 9., 10.],
       [11., 12.],
       [13., 14.],
       [15., 16.],
       [17., 18.],
       [19., 20.]])

In [961]:
llik_Kvec(a,b,lam,Xmat,Kvec)

51.36943404147456

In [786]:
def moments_beta(a, b, lam, K):
    C = len(a)
    A = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(a).reshape(1,len(a)))
    B = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(b).reshape(1,len(b)))
    # np.repeat: repeat each element
    # np.tile: repeat entire np.array
    mm = f_beta(np.add(A, np.repeat(np.linspace(0, K, K + 1), C).reshape(K+1, C)), B) / f_beta(A, B)
    L = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(lam).reshape(1, len(lam)))
    value = np.sum(np.multiply(mm, L), axis = 1)[1:]
    return value

In [464]:
moments_beta(a, b, lam, K)

array([0.40243243, 0.27710009, 0.21900677, 0.18462762, 0.16156419])

In [490]:
np.random.normal(size = 5)

8103.083927575384

In [524]:
norm.cdf(0.9)

0.8159398746532405

In [ ]:
def initial_values(rawmom, Cmix):
    if (Cmix==1):
        m1  = rawmom[0]
        q2  = rawmom[1]/rawmom[0]
        res = np.linalg.solve(np.column_stack((np.array([m1 - 1, q2 - 1]),np.array([m1, q2]))),np.array([0,1-q2]))
        if (np.sum(res > 0) == 2):
            return res
        if (np.sum(res > 0) == 0):
            return -res
        if (np.sum(res > 0) == 1):
            return m1/(1 - m1)
  
  
    if (Cmix>=2):
        K = 3 * Cmix - 1
        empmom = rawmom[0:K]
        init = [np.log([1, 
                        0.2, 
                        np.exp(np.random.normal(size = Cmix-1) + 0), # Changed the original code here which was: Cmix - 2
                        1,
                        5,
                        np.exp(np.random.normal(size = Cmix-1))]), # Changed the original code here which was: Cmix - 2
                [norm.ppf(1/Cmix)] * (Cmix-1)]

        #Reparameterized
        def fnmin(x):
            value = np.sum((moments_beta(np.exp(x[0:Cmix]),
                                 np.exp(x[(Cmix):(2*Cmix)]),
                                 np.array(norm.cdf(x[(2*Cmix):K]),1-np.sum(norm.cdf(x[(2*Cmix):K]))),
                                 K)
                        -empmom)**2)
            return value

        def grmin(x):
            a   = np.exp(x[0:Cmix]) #x[1:Cmix]
            b   = np.exp(x[(Cmix):(2*Cmix)]) #x[(Cmix+1):(2*Cmix)]
            lam = np.array(norm.cdf(x[(2*Cmix):(3*Cmix-1)]),1-sum(norm.cdf(x[(2*Cmix):(3*Cmix-1)])))
            #c(x[(2*Cmix+1):(3*Cmix-1)],1-sum(x[(2*Cmix+1):(3*Cmix-1)]))
            A = np.tile(a, K).reshape(K, len(a))
            B = np.tile(b, K).reshape(K, len(b))
            Kmat = np.repeat(range(1, K+1), Cmix).reshape(K, Cmix)
            mm = f_beta(A+Kmat,B)/f_beta(A,B)
            diga = digamma(A+Kmat)-digamma(A+B+Kmat)+digamma(A+B)-digamma(A)
            digb = digamma(B)-digamma(A+B+Kmat)+digamma(A+B)-digamma(B)
            L = np.tile(lam, K).reshape((K, len(lam)))

            deralpha = A*L*mm*diga #L*mm*diga
            derbeta  = B*L*mm*digb #L*mm*digb
            derlam   = np.delete(norm.pdf(norm.ppf(L)), mm.shape[1] + 1, axis = 1)*(mm[,-ncol(mm)]-mm[,ncol(mm)]%x%t(rep(1,ncol(mm)-1)))
            return( as.numeric(2*(moments_beta(a,b,lam,K)-empmom)%*%cbind(deralpha,derbeta,derlam)) )
                             

        res=optim(init,fn=fnmin,gr=grmin,method="BFGS",
        control=list(maxit=1000,trace=0))

        return(c(exp(res$par[1:(2*Cmix)]),norm.cdf(res$par[(2*Cmix+1):(length(res$par))])))

In [ ]:
def synthesis_moments(Xmat,Kvec,mommax=10):
    
    
    
  synthesis.moments <- function(Xmat,Kvec,mommax=10){
  if (!is.matrix(Xmat)){
      sumX = sum(Xmat)
      if (Kvec <= 20){
          resm=counts.to.moments(Xmat[1:(Kvec+1)],Kvec)
      }
      else{
          resm=empirical.moments(Xmat[1:(Kvec+1)],Kvec)
      }
  }
  else{
      sumX = apply(Xmat,2,sum)
      Kv1 = intersect(1:20,Kvec); ran1=which(Kvec %in% Kv1)
      Kv2 = setdiff(Kvec,Kv1)   ; ran2=which(Kvec %in% Kv2)
      if ((length(Kv1)!=0)&(length(Kv2)==0)){
          resm=counts.to.moments(Xmat[1:(max(Kv1)+1),ran1],Kv1)
      }
      if ((length(Kv1)==0)&(length(Kv2)!=0)){
          resm=empirical.moments(Xmat[1:(max(Kv2)+1),ran2],Kv2)
      }
      if ((length(Kv1)!=0)&(length(Kv2)!=0)){
          cm=counts.to.moments(Xmat[1:(max(Kv1)+1),ran1],Kv1)
          em=empirical.moments(Xmat[1:(max(Kv2)+1),ran2],Kv2)
          resm=(cm*sum(sumX[ran1]*Kv1)+em*sum(sumX[ran2]*Kv2))/
              sum(sumX*Kvec)
      }
  }
  resm ;
}

# Main Function

In [1049]:
def betamix_estim(Xmat, Kvec, Cmax, initval = None):
  
    ## Xmat is the count matrix
    ## Make it a matrix, even if one column only
    if (type(Xmat) is not np.ndarray): 
        Xmat = np.array(Xmat).reshape((len(Xmat), 1))
    

    ## In the standard case, length(Kvec) = ncol(Xmat)
    ## This script also supports the case in which length(Kvec)<ncol(Xmat),
    ##   that is, when you want to restrict the estimation to a subset of the Ks
    ## number of rows >= max(Kvec)+2
    ## number of cols >= length(Kvec)
    ## each column has K+1 meaningful elements, other elements are negative
    ## for j=1...K+1, value(j) = nb of obs such that (X_K = j-1)

    ## Find K for each column
    whichK = np.apply_along_axis(lambda x: np.min(np.where(x < 0)), axis = 0, arr = Xmat) - 2
    # Returns a tuple, that's why it is needed to extract the first element
    goodcol = np.where(np.isin(whichK, Kvec))[0]
    resm = np.zeros(shape = ((np.max(Kvec)+1, len(goodcol))))
    
    # Needed to change from R code due to zero index based
    for jj in range(len(goodcol)):
        resm[0:(Xmat[:,goodcol[jj]]>=0).sum(),jj]=Xmat[np.where(Xmat[:,goodcol[jj]] >= 0),goodcol[jj]-1]
        resm[resm < 0] = 0
    
    Xmat = resm
    
    if (len(Kvec) == 1):
        Xmat = np.array(Xmat).reshape((Xmat.shape[0] * Xmat.shape[1], 1))
    

    ## Estimate moments from observed proportions 
    mm = [0.5, 0.4, 0.3, 0.2, 0.1, 0.2, 0.4, 0.3, 0.2, 0.8] # synthesis_moments(Xmat,Kvec)

    reslist = [] #list()

    ## 1 component
    # Compute good initial values for the likelihood
    Cmix=1
    if (initval is None): 
        init = np.array([0.5] * len(mm)) #initial.values(mm,Cmix)
        init = np.log(init)

    else:
        init = np.log(mm) # np.log(initval)
        
    # I realized that this is not necessary
    
    # Max likelihood
    #def aux_fun(x):
    #    return llik_Kvec(np.exp(x[0:Cmix]), np.exp(x[(Cmix):(2*Cmix)]), 1, Xmat, Kvec)
    
    #prem = minimize(fun    = aux_fun,
    #                x0     = init,
    #                method = 'BFGS')
    print(init)
    def fn2(x):
        return llik_Kvec(x[0:Cmix], x[(Cmix):(2*Cmix)], [1], Xmat, Kvec)

    reslist = minimize(fn2, 
                       np.exp(init), 
                       method = "Nelder-Mead")[0][1] # Point where minimum occur
    #reslist[[Cmix]]$hessian = numDeriv::hessian(fn2,reslist[[Cmix]]$par)
    print('It worked')

    if (Cmax==1):
        return(reslist)
    
    else:
    ## 2 components and more
    ## For the moment, initial.values() goes up to Cmax=2
        for Cmix in list(range(1,Cmax)):
            if (initval is None): 
                init = np.array([0.5] * len(mm)) #initial.values(mm,Cmix)
                init = np.log(init)

            else:
                init = np.log(initval)

             # Initial values
            init = c(log(init[1:(2*Cmix)]),init[(2*Cmix+1):(3*Cmix-1)])

            # This is not necessary
            #def aux_fun2(x):
            #    return llik.Kvec(exp(x[1:Cmix]),exp(x[(Cmix+1):(2*Cmix)]),c(pnorm(x[(2*Cmix+1):(3*Cmix-1)]),1-sum(pnorm(x[(2*Cmix+1):(3*Cmix-1)]))),Xmat,Kvec)

            #prem = optim(init,
            #             fn = aux_fun2,
            #             method="BFGS",
            #             hessian=FALSE)

            init = np.array(np.exp(init[0:(2*Cmix)]),norm.cdf(init[(2*Cmix):(3*Cmix-1)]))

            def aux_fun3(x):
                return llik_Kvec(x[0:Cmix],x[(Cmix):(2*Cmix)],c(x[(2*Cmix):(3*Cmix-1)],1-sum(x[(2*Cmix):(3*Cmix-1)])),Xmat,Kvec)

            reslist[Cmix] = minimize(aux_fun3, 
                                     init, 
                                     method = "Nelder-Mead",
                                     options = {'maxiter': 2})[0][1] # Point where minimum occur



            #reslist[[Cmix]]$hessian = numDeriv::hessian(fn2,reslist[[Cmix]]$par)
            #reslist[[Cmix]]$lr.pval = 1-pchisq(2*(reslist[[Cmix-1]]$value-reslist[[Cmix]]$value),3)


        return reslist


In [1041]:
def fn2(x):
    return llik_Kvec(x[0:Cmix], x[(Cmix):(2*Cmix)], [1], Xmat, Kvec)

In [1004]:
x = [4,5,7,6,8,9,10]

def aux_fun(x):
    return llik_Kvec(np.exp(x[0:Cmix]), np.exp(x[(Cmix):2*Cmix]), 1, Xmat, Kvec)

In [1050]:
os.chdir('C:\\Users\\renan\\Desktop\\RIDIR project\\Non_Parametric_Binomial_Rathelot_2017\\beta_mixture_approach_2012\\Archive')

In [1051]:
data = pd.read_csv('simple.csv')
data.head()

,X,K
0,0,13
1,1,41
2,0,43
3,3,41
4,0,60


In [1052]:
def vector_to_count(data, rangeK = None):
    
    # data is a pandas dataframe with columns X and K
    
    X = data['X']
    K = data['K']
    
    if (rangeK is None):
        rangeK = list(set(K.values))
    
    else:
        X = X[K.isin(rangeK)]
        K = K[K.isin(rangeK)]
    
    Pmat = np.full(shape = (max(K)+2, len(rangeK)), fill_value = -99)

    for j in range(len(rangeK)):
        Kj = rangeK[j]
        Xj = X[K == Kj]
        aux = Xj+1
        Pmat[0:(Kj+1),j] = np.bincount(aux, minlength=Kj+2)[1:] # tabulate(Xj+1,nbins=Kj+1)
    
    return {'Xmat': Pmat,
            'Kvec': rangeK}

In [1053]:
data_changed = vector_to_count(data, rangeK = None)
Xmat = data_changed['Xmat']
Kvec = data_changed['Kvec']

In [1055]:
test = betamix_estim(Xmat, Kvec, Cmax = 2)

[-0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718]
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 1, 'cumsum': 0}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 2, 'cumsum': 165.66217615382692}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 3, 'cumsum': 498.08616298928075}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 4, 'cumsum': 807.7268115724228}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 5, 'cumsum': 1147.8173409863127}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 6, 'cumsum': 1430.0758030445922}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 7, 'cumsum': 1706.054361495752}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 8, 'cumsum': 1965.4358312070372}
{'a': array([0.5]), 'b': array([0.5]), 'lam': [1], 'Kvec[j]': 9, 'cumsum': 2277.7805527784813}
{'a': array([0.5]), 'b': array([0.5]), 'l

IndexError: index 87 is out of bounds for axis 1 with size 87